# HVNM Creep

**Hybrid Vitrimer-Nanocomposite Model — Retardation modes with nanoparticle amplification**

## Learning Objectives

- Understand retardation modes in filled vitrimer creep compliance J(t)
- Observe how nanoparticle strain amplification reduces creep
- Analyze elastic jump and long-time plateau 1/(G_P·X_I)
- Compare creep behavior across different nanoparticle volume fractions

**Runtime:** ~2 minutes

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax, verify_float64
jax, jnp = safe_import_jax()
verify_float64()

from rheojax.models import HVNMLocal
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## Theory: Creep Compliance with Nanoparticle Amplification

For a constant applied stress σ₀, the creep compliance is:

$$J(t) = \frac{\gamma(t)}{\sigma_0}$$

The HVNM model exhibits:

1. **Elastic jump** at t=0: J(0⁺) = 1/(G_P·X_I + G_E·X_E + G_D·X_D)
2. **Retardation modes**: Exponential relaxation from E and D networks
3. **Long-time plateau**: J(∞) → 1/(G_P·X_I) (permanent network with NP amplification)
4. **Vitrimer plastic creep**: Additional slow flow from TST-mediated bond exchange

**Key difference from HVM**: The strain amplification factors X_I, X_E, X_D reduce creep compliance:

- X(φ) = 1 + 2.5φ + 14.1φ² (Guth-Gold)
- Higher φ → larger X → stiffer response → less creep

The retardation times are:
- τ_E = η_E / (G_E·X_E) (E-network with NP amplification)
- τ_D = (G_D·X_D) / k_d_D (D-network with NP amplification)

In [ ]:
# Model setup
model = HVNMLocal()
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("phi", 0.1)       # Nanoparticle volume fraction
model.parameters.set_value("beta_I", 3.0)    # Immobilized fraction (I-network)
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-5)    # Activation volume [m³/mol]

# Compute strain amplification factors
phi = model.phi
X_I = 1.0 + 2.5*phi + 14.1*phi**2  # Guth-Gold

print("HVNM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  φ = {model.phi:.2f}")
print(f"  β_I = {model.beta_I:.1f}")
print(f"  T = {model.T:.0f} K")
print(f"  k_d_D = {model.k_d_D:.1f} 1/s")
print(f"  X_I(φ={phi:.2f}) = {X_I:.3f} (Guth-Gold amplification)")
print(f"  τ_D = (G_D·X_I)/k_d_D = {(model.G_D*X_I/model.k_d_D):.1f} s")
print(f"  Long-time compliance: 1/(G_P·X_I) = {1/(model.G_P*X_I):.2e} 1/Pa")


In [ ]:
# Creep simulation
t = np.logspace(-2, 4, 200)
sigma_0 = 100.0

print(f"Simulating creep at σ₀ = {sigma_0:.0f} Pa...")
result = model.simulate_creep(t, sigma_0=sigma_0, return_full=True)
J_t = np.array(result["strain"]) / sigma_0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Creep compliance
ax1.loglog(t, J_t, "k-", lw=2, label="J(t)")
ax1.axhline(1/(model.G_P*X_I), color="blue", ls=":", lw=1.5, 
            label=f"1/(G_P·X_I) = {1/(model.G_P*X_I):.2e}")
ax1.set_xlabel("t [s]")
ax1.set_ylabel("J(t) [1/Pa]")
ax1.set_title(f"Creep Compliance (φ = {phi:.2f})")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Creep strain
ax2.loglog(t, result["strain"], "k-", lw=2)
ax2.set_xlabel("t [s]")
ax2.set_ylabel("Strain γ")
ax2.set_title("Creep Strain (Monotonic)")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nCreep compliance range: {J_t.min():.2e} to {J_t.max():.2e} 1/Pa")
print(f"Final strain: {result['strain'][-1]:.4f}")

In [ ]:
# Nanoparticle volume fraction comparison
phi_values = [0.0, 0.1, 0.2]
colors = ['blue', 'green', 'red']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

print("\nComparing creep across different nanoparticle loadings...")
for phi_val, color in zip(phi_values, colors):
    model_temp = HVNMLocal()
    model_temp.parameters.set_value("G_P", 5000.0)
    model_temp.parameters.set_value("G_E", 3000.0)
    model_temp.parameters.set_value("G_D", 1000.0)
    model_temp.parameters.set_value("phi", phi_val)
    model_temp.parameters.set_value("beta_I", 3.0)
    model_temp.parameters.set_value("T", 350.0)
    model_temp.parameters.set_value("k_d_D", 10.0)
    model_temp.parameters.set_value("nu_0", 1e10)
    model_temp.parameters.set_value("E_a", 80e3)
    model_temp.parameters.set_value("V_act", 1e-5)
    
    r = model_temp.simulate_creep(t, sigma_0=sigma_0, return_full=True)
    J = np.array(r["strain"]) / sigma_0
    X = 1.0 + 2.5*phi_val + 14.1*phi_val**2
    
    ax1.loglog(t, J, "-", lw=1.5, color=color, label=f"φ = {phi_val:.1f} (X = {X:.3f})")
    ax2.loglog(t, r["strain"], "-", lw=1.5, color=color, label=f"φ = {phi_val:.1f}")
    print(f"  φ = {phi_val:.1f}: J(t_max) = {J[-1]:.2e} 1/Pa, γ(t_max) = {r['strain'][-1]:.4f}")

ax1.set_xlabel("t [s]")
ax1.set_ylabel("J(t) [1/Pa]")
ax1.set_title("Creep Compliance: φ Comparison")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

ax2.set_xlabel("t [s]")
ax2.set_ylabel("Strain γ")
ax2.set_title("Creep Strain: φ Comparison")
ax2.legend()
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

print("\n→ Higher φ → larger X → stiffer response → less creep")

## Key Takeaways

1. **Elastic jump**: Immediate elastic response at t=0 from all three networks (amplified by X factors)
2. **Retardation**: D-network and E-network relax with characteristic times τ_D and τ_E
3. **Long-time plateau**: Creep compliance approaches 1/(G_P·X_I) as only permanent network remains
4. **NP amplification reduces creep**: Higher φ → larger X → lower compliance → stiffer material
5. **Monotonic strain**: Creep strain grows monotonically without overshoot (contrast with startup)
6. **Vitrimer plasticity**: TST-activated bond exchange enables slow plastic flow beyond the plateau

## Physical Interpretation

The HVNM creep response reveals how nanoparticles reinforce vitrimer mechanics:

- **Immobilized chains**: β_I fraction experiences maximum amplification X_I ≈ 1.39 (at φ=0.1)
- **Reduced compliance**: J(∞) decreases by factor X_I compared to unfilled vitrimer
- **Guth-Gold scaling**: X(φ) = 1 + 2.5φ + 14.1φ² provides quadratic stiffening

At φ=0, the HVNM model reduces to HVM (no nanoparticle effects).

## Next Steps

- **05_hvnm_flow_curve.ipynb**: Steady-state flow and φ-independence at SS
- **06_hvnm_laos.ipynb**: Large-amplitude oscillatory shear and Payne effect
- **07_hvnm_limiting_cases.ipynb**: Factory methods and model comparisons
- Explore temperature dependence (T parameter)
- Fit experimental creep data using `.fit()` method